ISBN이 edition마다 다르지 않다는 사실을 알았음.. <br>
일단 같은 내용이니 같은 ISBN인 책 전부다 긁어 모은 최저가를 제시하자<br>

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

isbn = input('여기에 ISBN입력 : ') #9791191891287-희망1편, 9791192579504-희망2편, 9791156641131-절망편, 9791162241639-희망3편, 9788968484698-절판1편, 9788996991342-절판2편
html = urlopen("https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Used&KeyWord="+isbn+"&KeyRecentPublish=0&OutStock=0&ViewType=Detail&CustReviewCount=0&CustReviewRank=0&KeyFullWord="+isbn+"&KeyLastWord="+isbn+"&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&SuggestKeyWord=")
print("https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Used&KeyWord="+isbn+"&KeyRecentPublish=0&OutStock=0&ViewType=Detail&CustReviewCount=0&CustReviewRank=0&KeyFullWord="+isbn+"&KeyLastWord="+isbn+"&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&SuggestKeyWord=")
bs0bj = BeautifulSoup(html.read(),'html.parser') #html 전체 저장해놓고 있군..

여기에 ISBN입력 : 9791191891287
https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Used&KeyWord=9791191891287&KeyRecentPublish=0&OutStock=0&ViewType=Detail&CustReviewCount=0&CustReviewRank=0&KeyFullWord=9791191891287&KeyLastWord=9791191891287&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&SuggestKeyWord=


In [2]:
bo_used = bs0bj.findAll("a",{"class":"bo_used"}) #<a class="bo_used" href="~~">eBook</a> 같은 것 찾아줌
tmp_type, tmp_price, store_price =[], [], {}
used_store_html = []
for i in bo_used:
  if '원' in i.get_text(): # get_text하면 eBook만 출력함
    try:
      tmp_price.append(int(''.join(i.get_text()[:-1].split(','))))
    except:
      print("절판된 책")
      tmp_price.append(100000000) #절판됐으면 엄두도 못 낼 가격으로 해서 배제해버리자
  else:
    tmp_type.append(i.get_text())
  if '우주점' in i.get_text():
    print(i)
    used_store_html.append('https://www.aladin.co.kr' + i.attrs['href'])

print(tmp_type, tmp_price, used_store_html)

for i in range(len(tmp_type)):
  if tmp_type[i]=='새책' or '알라딘' in tmp_type[i]:
    try: #직배송 중고 있으면
      store_price['직배송'] = min(tmp_price[i],store_price['직배송'])
    except: #직배송 중고 없으면
      store_price['직배송'] = tmp_price[i]

#타입에 물건이 없으면 <span class='bo_used' ~~>로 바뀜

<a class="bo_used" href="/shop/UsedShop/wuseditemall.aspx?ItemId=312220558&amp;TabType=3">이 광활한 우주점 <br/><span class="bo_used_s">(15)</span> </a>
['새책', 'eBook', '알라딘 중고 (1)', '이 광활한 우주점 (15) ', '판매자 중고 (33)'] [13500, 10500, 10500, 10600, 6000] ['https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId=312220558&TabType=3']


In [3]:
print(used_store_html)
if used_store_html != []:
  for i in used_store_html:
    html = urlopen(i)
    bs0bj = BeautifulSoup(html.read(),'html.parser')
    price = bs0bj.findAll('span',{"class":"Ere_fs20 Ere_sub_pink"}) #가격 리스트를 전달 함
    seller = bs0bj.findAll('li',{"class":"Ere_store_name"}) # 매장 리스트를 전달함
      # findAll은 위에서부터 순서대로 찾아내서 같은 인덱스번호에 매장-가격 위치함
    for j in range(len(price)):
      print(seller[j].get_text(), end=' : ')
      print(price[j].get_text())
      for k in range(len(price)):
        if seller[k].get_text()[4:] in store_price: #같은 매장에서는 최저가만
          pass
        else:
          store_price[seller[k].get_text()[4:]] = int(''.join(price[k].get_text().split(",")))
else:
  print("중고매장에 책 없음")

['https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId=312220558&TabType=3']
중고매장잠실롯데월드타워점 : 10,600
중고매장부산덕천점 : 10,600
중고매장울산점 : 10,600
중고매장광주상무점 : 10,600
중고매장분당서현점 : 10,600
중고매장잠실새내역점 : 10,600
중고매장동탄2하나로마트점 : 10,600
중고매장신촌점 : 10,600
중고매장대학로점 : 10,600
중고매장목동점 : 10,600
중고매장수지점 : 10,600
중고매장신림점 : 10,600
중고매장목동점 : 10,600
중고매장신림점 : 10,600
중고매장건대점 : 10,600


In [4]:
try:
  if store_price['직배송']==100000000: #절판인데 알라딘 직배송 중고가 없다면
    del store_price['직배송']
except:
  pass
store_price

{'직배송': 10500,
 '잠실롯데월드타워점': 10600,
 '부산덕천점': 10600,
 '울산점': 10600,
 '광주상무점': 10600,
 '분당서현점': 10600,
 '잠실새내역점': 10600,
 '동탄2하나로마트점': 10600,
 '신촌점': 10600,
 '대학로점': 10600,
 '목동점': 10600,
 '수지점': 10600,
 '신림점': 10600,
 '건대점': 10600}